<a href="https://colab.research.google.com/github/MWaser/BurstIQ/blob/main/API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Upgrade Python Version if necessary

The BurstIQ API requires Python 3.11 or greater.  Check the version to see if upgrading is necessary.

In [2]:
!python --version

Python 3.10.12


If upgrading is necessary, run the next box.
Do not worry about the virtual machine warnings -- a notebook is already a virtual machine.

In [3]:
!sudo apt-get update -y
!sudo apt-get install python3.11

!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 2

!python --version

!sudo apt-get install python3-pip

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,309 kB]
Get:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,617 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://ppa.launchpadcontent.net/c2d4u.team/c2

# 2. Install BurstIQ Command Line (cliq2)

Grab Python Wheel Installation file from Github Repo

In [4]:
!curl -H 'Accept: application/vnd.github.v3.raw' -O \
  -L https://api.github.com/repos/clinical-squared/BurstIQ/contents/cliq2-1.0.0-py3-none-any.whl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 21896  100 21896    0     0  95892      0 --:--:-- --:--:-- --:--:-- 96035


Install from Wheel file

In [5]:
!pip3 install cliq2-1.0.0-py3-none-any.whl

Processing ./cliq2-1.0.0-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 KB 26.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 KB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.7/757.7 KB 69.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 KB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.5/162.5 KB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 KB 12.6 MB/s eta 0:00:00
  Created wheel for clint: filename=clint-0.5.1-py3-none-any.whl size=34473 sha256=9f5a7574ea8a248d399d558a1db325f7c4f60d7acedafe8d5480a3d

Verify that command line program is installed correctly

In [6]:
!cliq2 -h

usage: cliq2 [-h] [-p PROPS_FILE]
             {version,drop_customer,drop_sdz,update_customer,predict,update_dict,import_dbschema,export_dbschema,truncate_chain,load,query,create_smart,execute_smart}
             ...

Command Line IQ (CLIQ) working with LifeGraph (aka GraphChain): Performs many of the admin-centric
commands necessary to support/manage a customer and its SDZs.

positional arguments:
  {version,drop_customer,drop_sdz,update_customer,predict,update_dict,import_dbschema,export_dbschema,truncate_chain,load,query,create_smart,execute_smart}
                        type "<cmd> --help" for help on specific command
    version             version number
    drop_customer       drop a customer by short name, BIQ_ADMIN only
    drop_sdz            drop a customer's sdz by short name, BIQ_ADMIN only
    update_customer     upserts a customer JSON configuration file, BIQ_ADMIN only
    predict             uses a data file to predict the dictionary and mapping
    update_dict      

In [7]:
!cliq2 update_customer -h

usage: cliq2 update_customer [-h] --file FILE

options:
  -h, --help            show this help message and exit
  --file FILE, -f FILE  the file path for the input


# 3. Build the Config file (cliq2.yml)

Documentation is at https://admin.burstiq.com/docs/pages/metadata_tools/cliq2/cliq2_desc.html#configuration

Example cliq2.yml:
> biq_admin:
>>  username: burstiq.admin  
>>  password: XXXXXXXXXXXXXXXXXX  
>>  server: https://keycloak.app.burstiq.com/auth  
>>  client_id: aaaaaaaaaaaa  
>>  realm: bbbbbbbbbbbb

> sdz_admin:
>>  username: john.wayne  
>>  password: XXXXXXXXXXXXXXXXXX  
>>  server: https://keycloak.app.burstiq.com/auth  
>>  client_id: burst  
>>  realm: { customer_shortname }  

> graphchain:
>>  server: https://app.burstiq.com  
>>  customer: { customer_shortname }  
>>  sdz: { customer_sdz_name }  

Use secrets for username, password, customer_shortname and customer_sdz name (the key on the left menu) to generate cliq.yml <br />
<b>IMPORTANT!!! </b> Be sure to delete the results of the following step before saving or all your secret information will be in the save file!

In [ ]:
from google.colab import userdata
yml = open(r"cliq2.yml", "w")
yml.write("sdz_admin:\n");
yml.writelines("   username: " + userdata.get('username') + "\n");
yml.writelines("   password: " + userdata.get('password') + "\n");
yml.writelines("   server: https://keycloak.app.burstiq.com/auth\n");
yml.writelines("   client_id: burst\n");
yml.writelines("   realm: " + userdata.get('customer_shortname') + "\n");
yml.writelines("graphchain:\n");
yml.writelines("   server: https://app.burstiq.com\n");
yml.writelines("   customer: " + userdata.get('customer_shortname') + "\n");
yml.writelines("   sdz: " + userdata.get('customer_sdz_name') + "\n");
yml.close();

!cat cliq2.yml
